# 5. Fq2peaks
## Megatid required
Call ANY type of peak from reads stored in fq.gz files.
Can call **peaks** for:
* **factor**:
Peak finding for single contact or focal ChIP-Seq experiments or DNase-Seq.  This type of analysis is useful for transcription factors, and aims to identify the precise location of DNA-protein contact.  This type of peak finding uses a FIXED width peak size, which is automatically estimated from the Tag Autocorrelation.

* **histone**: 
Peak finding for broad regions of enrichment found in ChIP-Seq experiments for various histone marks.  This analysis finds variable-width peaks.

* **super**: 
Find Super Enhancers in your data

* **groseq**:
De novo transcript identification from strand specific GRO-Seq.  This attempts to identify transcripts from nascent RNA sequencing reads.

* **tss**
Identification of promoter/TSS from 5'RNA-Seq/CAGE or 5'GRO-Seq data.

* **dnase**
Adjusted parameters for DNase-Seq peak finding.

* **mC** 
DNA methylation analysis

## <a href="http://homer.ucsd.edu/homer/ngs/peaks.html"> Homer peak calling tutorial </a> for more info

In [7]:
#Define data folders, data, and sample name
#remember that fasta should have built bowtie2 indices
annotation="/input_dir/mm9"
tmp="/input_dir/"
input_dir="/input_dir/"
output_dir="/output_dir/"
input_fq_1="h3k27ac_mesc_5M.fq.gz"
input_fq_2=""
sample_name="mesc_h3k27ac_5M"

# Align reads to reference genome

In [9]:
#align and sort / filter bam using Bowtie2 and "sensitive" settings
#paired end
if [ -z "$input_dir""$input_fq_2" ]; then
    echo "Aligning using bowtie2 w/ paired end"
    bowtie2 -p 8 --sensitive -x  $annotation \
        -1 $input_dir$input_fq_1 -2 $input_dir$input_fq_2 | samtools view -bS - > $tmp$sample_name".bam"
else
    echo "Aligning using bowtie2 w/ single end"
    bowtie2 -p 8 --sensitive -x  $annotation \
        -U $input_dir$input_fq_1 | samtools view -bS - > $tmp$sample_name".bam"
fi

#sort index and filter for canonical chromosomes
samtools sort -@ 10 $tmp$sample_name".bam" -o $tmp$sample_name"_sorted.bam"
samtools index "$tmp$sample_name""_sorted.bam"
samtools view $tmp$sample_name"_sorted.bam" -hu chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chrX > "$output_dir$sample_name""_filt_sorted.bam"
samtools index "$output_dir$sample_name""_filt_sorted.bam"

Aligning using bowtie2 w/ single end
5000000 reads; of these:
  5000000 (100.00%) were unpaired; of these:
    704720 (14.09%) aligned 0 times
    3416197 (68.32%) aligned exactly 1 time
    879083 (17.58%) aligned >1 times
85.91% overall alignment rate


In [ ]:
#Take a peek at the bam
samtools view "$output_dir$sample_name""_filt_sorted.bam"| head
#get alignment stats
samtools flagstat "$output_dir$sample_name""_filt_sorted.bam"
samtools idxstats "$output_dir$sample_name""_filt_sorted.bam"

# Using Homer to call peaks
Homer is a pretty sweet collection of mainly perl scripts which can handle next-gen sequencing data to do things such as call peaks, below we use it to call different types of peaks, more information and a lot of knowledge is <a href="http://homer.ucsd.edu/homer/ngs/peaks.html"> Homer peak calling tutorial </a>  

In [ ]:
#Homer run
#make tag directory file for Homer using aligned .bam file
makeTagDirectory $tmp/"$sample_name"_homer_tags "$output_dir$sample_name""_filt_sorted.bam"

In [3]:
#find Chromatin accessibility peaks 
findPeaks $tmp/"$sample_name"_homer_tags -style dnase -o auto
pos2bed.pl "$tmp""$sample_name"_homer_tags/peaks.txt > "$output_dir$sample_name""_open_peaks.bed"

	Will parse file: /output_dir/mesc_atac_5M_filt_sorted.bam

	Creating directory: /input_dir//mesc_atac_5M_homer_tags and removing existing *.tags.tsv

	Treating /output_dir/mesc_atac_5M_filt_sorted.bam as a bam file
	Reading alignment file /output_dir/mesc_atac_5M_filt_sorted.bam

	Optimizing tag files...
	Estimated genome size = 2638916841
	Estimated average read density = 0.000322 per bp
	Total Tags = 849746.0
	Total Positions = 1534940
	Average tag length = 124.5
	Median tags per position = 0 (ideal: 1)
	Average tags per position = 0.089
	Fragment Length Estimate: 174
	Peak Width Estimate: 607
	Autocorrelation quality control metrics:
		Same strand fold enrichment: 1.6
		Diff strand fold enrichment: 4.4
		Same / Diff fold enrichment: 0.3

		Guessing sample is ChIP-Seq - uneven enrichment between same strand and
		different strands - may have problems such as clonal amplification.

	Fragment Length = 1
	Total Tags = 849746.0
	Tags per bp = 0.000425
	Max tags per bp set automatically 

In [ ]:
#find enhancers and super enhancers
findPeaks $tmp/"$sample_name"_homer_tags -style super -typical  $tmp/"$sample_name"_homer_tags/enh_peaks.txt -o auto
#convert regular enhancer peak file
pos2bed.pl $tmp/"$sample_name"_homer_tags/enh_peaks.txt > "$output_dir$sample_name""_enh_peaks.bed"
#save peaks as bed file in output directory
pos2bed.pl "$tmp""$sample_name"_homer_tags/peaks.txt > "$output_dir$sample_name""_se_peaks.bed"

## Use MACS2 to call peaks and NucleoATAC to call nucleosomes 

In [4]:
#Switch to python2.7 for MACS2 and NucleoATAC
source activate py27
#Call acessibility peaks, and using those peaks define area around to resolve nucleosomes
macs2 callpeak -t "$output_dir$sample_name""_filt_sorted.bam" --nomodel --shift -37 -f BAMPE \
    --extsize 73 --broad --keep-dup all -n $tmp"$sample_name""_MACS"
bedops --range 500:500 --everything $tmp"$sample_name""_MACS_peaks.broadPeak" | bedtools merge -i - > $tmp"$sample_name""_MACS_merged.bed"
nucleoatac run --bed $tmp"$sample_name""_MACS_merged.bed" --bam $output_dir"$sample_name""_filt_sorted.bam" --fasta "$annotation"".fa" --out $tmp"$sample_name""_nucATAC" --cores 8
source deactivate py27

(py27) (py27) INFO  @ Tue, 30 May 2017 23:22:21: 
# Command line: callpeak -t /output_dir/mesc_atac_5M_filt_sorted.bam --nomodel --shift -37 -f BAMPE --extsize 73 --broad --keep-dup all -n /input_dir/mesc_atac_5M_MACS
# ARGUMENTS LIST:
# name = /input_dir/mesc_atac_5M_MACS
# format = BAMPE
# ChIP-seq file = ['/output_dir/mesc_atac_5M_filt_sorted.bam']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff for narrow/strong regions = 5.00e-02
# qvalue cutoff for broad/weak regions = 1.00e-01
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is on
# Paired-End mode is on
 
INFO  @ Tue, 30 May 2017 23:22:21: #1 read fragment files... 
INFO  @ Tue, 30 May 2017 23:22:21: #1 read treatment fragments... 
INFO  @ Tue, 30 May 2017 23:22:35:  1000000 
INFO  @ Tue, 30 May 2017 23:22:39: #1 mean fragment size is determined as 229 bp from treatment 
INFO  @